# 4. The Standard Model

In this chapter, we'll go through all fields in the standard model, explain their meaning and the default functions that calculate them.

In [1]:
from tripodpy import Simulation
import numpy as np

In [2]:
sim = Simulation()

In [3]:
sim.ini.grid.Nr = 20 
sim.makegrids()
sim.initialize()

By default, the frame object of `TwoPopPy` consists of four groups for **dust**, **gas**, **grid**, and **stellar** parameters, a field for the **time**, which is the integration variable, an **integrator** object, and a **writer** object.

In [4]:
sim

tripodpy
--------
    components   : Group (components)
    dust         : Group (Dust quantities)
    gas          : Group (Gas quantities)
    grid         : Group (Grid quantities)
    star         : Group (Stellar quantities)
  -----
    t            : IntVar (Time [s]), Integration variable
  -----
    Integrator   : Integrator (Default integrator)
    Writer       : Writer (HDF5 file format using h5py)

## Dust

In [5]:
sim.dust

Group (Dust quantities)
-----------------------
    backreaction : Group (Backreaction coefficients)
    boundary     : Group (Boundary conditions)
    delta        : Group (Mixing parameters)
    f            : Group (Fudge factors)
    Fi           : Group (Fluxes)
    p            : Group (Probabilities)
    q            : Group (Distribution exponents)
    S            : Group (Sources)
    s            : Group (Characteristic particle sizes)
    v            : Group (Velocities)
  -----
    a            : Field (Particle sizes in cm: [a0, fudge * a1, a1, 0.5 * amax, amax])
    D            : Field (Diffusivity [cm²/s])
    eps          : Field (Dust-to-gas ratio)
    fill         : Field (Filling factor)
    H            : Field (Scale heights [cm])
    m            : Field (Particle mass [g])
    qrec         : Field (reconstructed distribution exponent)
    rho          : Field (Midplane mass density per mass bin [g/cm³])
    rhos         : Field (Solid state density [g/cm³])
  

### `Simulation.dust.backreaction`

In [6]:
sim.dust.backreaction

Group (Backreaction coefficients)
---------------------------------
    A            : Field (Pull factor)
    B            : Field (Push factor)
  -----

The backreaction describes the hydrodynamic influence the dust has on the gas. Numerically, it consists of two fields `A` and `B` of shape `(Simulation.grid.Nr,)` that describe the pull or rather the push the dust excerts on the gas respectively.

The details of this mechanism are described in [Gárate et al. (2019)](https://doi.org/10.3847/1538-4357/aaf4fc).

Backreaction modifies the radial gas velocity as follows

$v_\mathrm{g} = Av_\mathrm{visc} + 2B\eta v_\mathrm{K}$.

In the standard model, we have `A=1` and `B=0` everywhere, i.e. backreactions are not active.

In [7]:
sim.dust.backreaction.A

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

In [8]:
sim.dust.backreaction.B

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Depending on the type of backreaction that we want to model, we have to provide functions for `A` and `B`. Please also have a look at Appendix A of [Gárate et al. (2019)](https://doi.org/10.3847/1538-4357/aaf4fc) for examples.

### `Simulation.dust.boundary`

In [9]:
sim.dust.boundary

Group (Boundary conditions)
---------------------------
    inner        : Boundary
    outer        : Boundary
  -----

By default, the inner dust boundary is set to constant gradient and the outer boundary to floor value.

In [10]:
sim.dust.boundary.inner

Constant gradient

In [11]:
sim.dust.boundary.outer

Value

The outer boundary therefore stores an array with the floor values of all particle masses.

The boundary conditions can be changed via `setcondition()`.

In [12]:
help(sim.dust.boundary.outer.setcondition)
sim.dust.boundary.inner.setcondition("grad", -1)

Help on method setcondition in module dustpy.utils.boundary:

setcondition(condition, value=None) method of dustpy.utils.boundary.Boundary instance
    Function to set boundary condition.

    Parameters
    ----------
    condition : string
        Type of boundary conditon:
            - "const_grad" : constant gradient
            - "const_pow" : constant power law
            - "const_val" : constant value
            - "val" : custom value
            - "grad" : custom gradient
            - "pow" : custom power law with set exponent
            - None : Don't impose boundary condition (default)
    value : float or array, optional, default : None
        Value if needed for boundary condition



### `Simulation.dust.delta`

In [13]:
sim.dust.delta

Group (Mixing parameters)
-------------------------
    rad          : Field (Radial mixing parameter)
    turb         : Field (Turbulent mixing parameter)
    vert         : Field (Vertical mixing parameter)
  -----

The $\delta$ parameters control the mixing of dust particles along vertical and radial directions and turbulent mixing. We can look at them as similar to the turbulent $\alpha$ parameter for the gas. By default, they will also have the same value as $\alpha$ as given by `Simulation.ini.gas.alpha`.

#### `Simulation.dust.delta.rad`

In [14]:
sim.dust.delta.rad

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]

$\delta_\mathrm{rad}$ will be used to calculate the radial dust diffusion.

#### `Simulation.dust.delta.turb`

In [15]:
sim.dust.delta.turb

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]

$\delta_\mathrm{turb}$ will be used to calculate the turbulent collision velocities of the dust particles.

#### `Simulation.dust.delta.vert`

In [16]:
sim.dust.delta.vert

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]

$\delta_\mathrm{vert}$ will be used to calculate the vertical dust scale heights.

### `Simulation.dust.Fi`

In [17]:
sim.dust.Fi

Group (Fluxes)
--------------
    adv          : Field (Advective flux [g/cm/s])
    diff         : Field (Diffusive flux [g/cm/s])
    tot          : Field (Total flux [g/cm/s])
  -----

This is a group of fluxes through the radial grid interfaces for every particle mass of shape `(Simulation.grid.Nr+1, 2)`.

**Attention:** When using implicit dust integration, the interface fluxes are calculated in retrospect after the new dust surface density was found. Changing any `Simulation.dust.Fi` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.adv`

This is the advective flux calculated by $F_\mathrm{adv} = v_\mathrm{d}\Sigma_\mathrm{d}$.

**Attention:** When using implicit dust integration, the advective interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.adv` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.diff`

This is the diffusive flux calculated by $F_\mathrm{diff} = -D\Sigma_\mathrm{g}\nabla\frac{\Sigma_\mathrm{d}}{\Sigma_\mathrm{gas}}$ for every particle species separately. The diffusive fluxes at the grid boundaries are set to zero to avoid instabilities.

**Attention:** When using implicit dust integration, the diffusive interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.diff` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.Fi.tot`

This is the total flux through the radial grid interfaces $F_\mathrm{tot} = F_\mathrm{adv} + F_\mathrm{diff}$.

**Attention:** When using implicit dust integration, the total interface fluxes are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.Fi.tot` does not influence the simulation. It is only given for data analysis.

### `Simulation.dust.p`

In [18]:
sim.dust.p

Group (Probabilities)
---------------------
    driftfrag    : Field (Transition function from drift to turbulence)
    frag         : Field (Fragmentation probability)
    fragtrans    : Field (transition probability between fragmentation regimes)
    stick        : Field (Sticking probability)
  -----

These are the fragmentation and sticking probability of certain particle collisions.  
Each cell has a scalar probability, e.g the shape of p.frag is `(Simulation.grid.Nr)`.

Collisions between particles of sizes $0.4*a_\text{max}$ and $a_\text{max}$ are representative for the entire dust distribution. The characteristic fragmentation probability at radial grid cell `ir=15` is therefore given by

In [19]:
ir = 15
sim.dust.p.frag[ir]

np.float64(8.08062007266603e-09)

#### `Simulation.dust.p.frag`

`TriPoD` assumes that the relative collision velocities follow the Maxwell-Boltzmann distribution

$f\left(\Delta v\right) = \sqrt{\frac{54}{\pi}}\frac{\Delta v^{2}}{v_\mathrm{rms}^3} \exp \left[-\frac{3}{2} \frac{\Delta v^2}{v_\mathrm{rms}^2} \right]$.

The collision kernel for fragmentation is then given by

$K_\mathrm{f} = \int\limits_{v_\mathrm{frag}}^\infty \sigma_\mathrm{geo} \Delta v f\left(\Delta v\right) \mathrm{d}\Delta v$

by counting all collisions above the fragmentation velocity.

The fragmentation probability is then given by

$p_\mathrm{f} = \frac{K_\mathrm{f}}{\sigma_\mathrm{geo} \Delta \bar{v}}$

with the mean velocity of the Maxwell-Boltzmann distribution

$\Delta \bar{v} = \sqrt{\frac{8\pi}{2}}v_\mathrm{rms}$.

The root-mean-square velocities $v_\mathrm{rms}$ are stored in `Simulation.dust.v.rel`.

#### `Simulation.dust.p.stick`

This is the sticking probability given by $p_\mathrm{s} = 1 - p_\mathrm{f}$.

No bouncing include yet.

### `Simulation.dust.S`

In [20]:
sim.dust.S

Group (Sources)
---------------
    coag         : Field (Coagulation sources [g/cm²/s])
    compo        : Field (Sources due to composition changes [g/cm²/s])
    ext          : Field (External sources [g/cm²/s])
    hyd          : Field (Hydrodynamic sources [g/cm²/s])
    smax_hyd     : Field (Total sources [g/cm²/s])
    tot          : Field (Total sources [g/cm²/s])
  -----

These are the source terms of the dust of shape `(Simulation.grid.Nr, 2)` used to integrate the time evolution of the dust.

#### `Simulation.dust.S.coag`

The source terms from dust coagulation given by 

$S_{\mathrm{coag},\,0 \rightarrow 1} = \dot{\Sigma}_{0 \rightarrow 1} - \dot{\Sigma}_{1 \rightarrow 0} =  \frac{\Sigma_0 \Sigma_1 \sigma_{01} \Delta v_{01}}{m_1 \sqrt{2 \pi \left(H_0^2 + H_1^2\right)}} - \frac{\Sigma_1^2 \sigma_{11} \Delta v_{11}}{m_1 2 \sqrt{\pi} H_1} \mathcal{F} = - S_{\mathrm{coag},\,1 \rightarrow 0}$

with $\mathcal{F} = \sqrt{\frac{2H_1^2}{H_0^2 + H_1^2}} \frac{\sigma_{01}\Delta v_{01}}{\sigma_{11}\Delta v_{11}}\left(\frac{s_{max}}{s_{min}}\right)^{-\left(\xi'+4\right)}$ and $\xi' = \Pi_{frag}\xi_{frag} + \Pi_{stick}\xi_{stick}$

#### `Simulation.dust.S.ext`

External source terms. These are by default set to zero, i.e. no external sources.

In [21]:
sim.dust.S.ext

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]

#### `Simulation.dust.S.hyd`

These are the hydrodynamic source terms. At grid cell $i$ they are calculated by

$S_{\mathrm{hyd},\,i} = 2\frac{\left( r_{i-\frac{1}{2}}F_{i-\frac{1}{2}}-r_{i+\frac{1}{2}}F_{i+\frac{1}{2}} \right)}{r_{i+\frac{1}{2}}^2 - r_{i-\frac{1}{2}}^2}$.

**Attention:** When using implicit dust integration, the hydrodynamic source terms are calculated in retrospect after the new dust surface density was found. Changing `Simulation.dust.S.hyd` does not influence the simulation. It is only given for data analysis.

#### `Simulation.dust.S.tot`

These are the total source terms given by $S_\mathrm{tot} = S_\mathrm{coag} + S_\mathrm{ext} + S_\mathrm{hyd}$.

### `Simulation.dust.s`

In [22]:
sim.dust.s

Group (Characteristic particle sizes)
-------------------------------------
    boundary     : Group (boundary conditions of smax)
  -----
    lim          : Field (Limiting size for shrinking)
    max          : Field (Maximum particle size)
    min          : Field (Minimum particle size)
    sdot_coag    : Field (coagulation source term for amax [cm/s])
    sdot_shrink  : Field (shrinkage source term for amax [cm²/s])
  -----

These are specific particle sizes of shape `(Simulation.grid.Nr)` characterizing the dust distribution.

#### `Simulation.dust.s.max`

This is the maximum particle size of the dust distribution. By default, it is initialized as `Simulation.ini.dust.aIniMax`.

In [23]:
sim.dust.s.max

[1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 1.00000000e-04
 1.00000000e-04 1.00000000e-04 1.00000000e-04 7.84313275e-05
 7.84313275e-05 7.84313275e-05 7.84313275e-05 7.84313275e-05]

Over the course of the simulation, it is evolved in time according to the source term

$\dot{s}_{max} = \frac{\rho_d}{\rho_s}\Delta v \left[\frac{1-\left(\frac{v_{frag}}{\Delta v}\right)^s}{1+\left(\frac{v_{frag}}{\Delta v}\right)^s}\right]$

describing its growth and decline due to sticking and fragmentation.

#### `Simulation.dust.s.min`

This is the minimum particle size of the dust distribution, which remains constant over time. It is by default set to $0.1 *$ `Simulation.ini.dust.aIniMax`.

In [24]:
sim.dust.s.min

[5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05
 5.22875516e-05 5.22875516e-05 5.22875516e-05 5.22875516e-05]

#### `Simulation.dust.s.lim`

 This is the minimum particle size of the dust distribution, which form the gap shrikage term as described in Pfeil+ 2024

In [25]:
sim.dust.s.lim

0.0001

#### `Simulation.dust.s.boundary`

the boundary for the maximal particle size works the same was as for the dust but sets the boundary condition for `sim.dust.s.max*sim.dust.Sigma[:,1]` which is the quantitiy being advected by the solver

In [26]:
sim.dust.s.boundary.inner

Constant gradient

In [27]:
sim.dust.s.boundary.outer

Value

### `Simulation.dust.v`

In [28]:
sim.dust.v

Group (Velocities)
------------------
    rel          : Group (Relative velocities)
  -----
    driftmax     : Field (Maximum drift velocity [cm/s])
    frag         : Field (Fragmentation velocity [cm/s])
    rad          : Field (Radial velocity [cm/s])
    rad_flux     : Field (Radial velocity modified to claulate the proper flux[cm/s])
  -----

These are some dust related velocities the simulation needs for execution. 

#### `Simulation.dust.v.rel`

In [29]:
sim.dust.v.rel

Group (Relative velocities)
---------------------------
    azi          : Field (Relative azimuthal velocity [cm/s])
    brown        : Field (Relative Brownian motion velocity [cm/s])
    rad          : Field (Relative radial velocity [cm/s])
    tot          : Field (Total relative velocity [cm/s])
    turb         : Field (Relative turbulent velocity [cm/s])
    vert         : Field (Relative vertical settling velocity [cm/s])
  -----

These are the different sources of relative particle collision velocities used by the coagulation algorithm.  
They are used for two different reasons:
1. To calculated the collision rates
2. To determine the outcome of a collision

#### `Simulation.dust.v.rel.azi`

Relative collision velocity caused by a differential azimuthal drift of particles with different Stokes numbers calculated by

$v_{\mathrm{rel},\,\mathrm{azi}} = \left| v_{\mathrm{drift},\,\mathrm{max}} \cdot \left( \frac{1}{1+\mathrm{St}_i^2} - \frac{1}{1+\mathrm{St}_j^2} \right) \right|$.

##### `Simulation.dust.v.brown`

Relative collision velocity of particles caused by Brownian motion calculated with

$v_{\mathrm{rel},\,\mathrm{brown}} = \sqrt{ \frac{8k_\mathrm{B}T\left(m_i + m_j \right)}{\pi m_i m_j} }$.

Since this expression is diverging for very small particle masses, the relative velocity is capped to a maximum value of the sound speed $c_\mathrm{s}$. For very small particles this can still be larger than the fragmentation velocity and can cause unwanted fragmentation in the simple coagulation algorithm implemented in the default model.

##### `Simulation.dust.v.rel.rad`

Relative collision velocity caused by differential radial drift.

$v_{\mathrm{rel},\,\mathrm{rad}} = \left| v_{\mathrm{rad},\,i} - v_{\mathrm{rad},\,j} \right|$.

##### `Simulation.dust.v.rel.tot`

Total relative velocities calculated by using the root mean square of all individual velocity sources.

$v_{\mathrm{rel},\,\mathrm{tot}} = \sqrt{v_{\mathrm{rel},\,\mathrm{azi}}^2 + v_{\mathrm{rel},\,\mathrm{brown}}^2 + v_{\mathrm{rel},\,\mathrm{rad}}^2 + v_{\mathrm{rel},\,\mathrm{turb}}^2 + v_{\mathrm{rel},\,\mathrm{vert}}^2}$.

##### `Simulation.dust.v.rel.turb`

Relative velocities caused by turbulent motion of the particles. This follows the prescription of [Ormel & Cuzzi (2007)](https://doi.org/10.1051/0004-6361:20066899).  
It uses `Simulation.dust.delta.turb` instead of `Simulation.gas.alpha` to calculate the velocities.

##### `Simulation.dust.v.rel.vert`

Relative collision velocities caused by differential vertical settling of particles.

$v_{\mathrm{rel},\,\mathrm{vert}} = \left| h_i \min \left( \mathrm{St}_i,\,\frac{1}{2}\right) - h_j \min \left( \mathrm{St}_j,\,\frac{1}{2}\right) \right| \cdot \Omega_\mathrm{K}$.

This prescription is taken from [Birnstiel et al. (2010)](https://doi.org/10.1051/0004-6361/200913731) and follows [Dullemond & Dominik (2004)](https://doi.org/10.1051/0004-6361:20040284).

#### `Simulation.dust.v.driftmax`

This is the maximum drift velocity a particle of $\mathrm{St} = 1$ can have.

$v_{\mathrm{drift},\,\mathrm{max}} = \frac{1}{2} B v_\mathrm{visc} - A \eta v_\mathrm{K}$.

See `Simulation.dust.backreaction` for details.

#### `Simulation.dust.v.frag`

Fragmentation velocities of shape `(Simulation.grid.Nr)`. By default this is set by the value of `Simulation.ini.dust.vfrag`.

In [30]:
sim.dust.v.frag

[100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100.]

#### `Simulation.dust.v.rad`

Radial velocities of the dust particles.

$v_\mathrm{rad} = \left( v_\mathrm{g} + 2 v_{\mathrm{drift},\,\mathrm{max}}\mathrm{St} \right) \frac{1}{\mathrm{St}^2+1}$

#### `Simulation.dust.v.rad_flux`

Radial velocities of the dust particles used for the evolution of the surface densities which includes the callibration factor $f_{drift}$.

$v_\mathrm{rad} = f_{drift} \left( v_\mathrm{g} + 2 v_{\mathrm{drift},\,\mathrm{max}}\mathrm{St} \right) \frac{1}{\mathrm{St}^2+1}$

### `Simulation.dust.qrec`


This is the exponent of the current particle size distribution calculated via

$q = \frac{\log\left(\Sigma_1/\Sigma_0\right)}{\log\left(s_{max}/s_{int}\right)} - 4$

In [31]:
sim.dust.qrec

[-3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5 -3.5
 -3.5 -4.  -4.  -4.  -4.  -4. ]

These are exponents of the particle size distribution $\sigma\left(a\right) \propto a^\xi$ of shape `(Simulation.grid.Nr)`.

#### `Simulation.dust.q.eff`

This is the exponent of the particle size distribution strives for in the equilibrium

In [32]:
sim.dust.q.eff

[-3.         -3.         -3.         -3.         -3.         -3.
 -3.         -3.         -3.         -3.         -3.         -3.
 -3.         -3.         -3.         -3.00000001 -3.00081719 -3.75
 -3.75       -3.        ]

#### `Simulation.dust.q.frag`

This is the exponent of the particle size distribution in the fragmentation limited case and is between `-3.5` and `-3.75` depending on the regime (see [Pfeil+ 2024](https://ui.adsabs.harvard.edu/abs/2024A&A...691A..45P))

In [33]:
sim.dust.q.frag

[-3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.75       -3.75       -3.75       -3.75       -3.75       -3.75
 -3.74999999 -3.74999996 -3.74999966 -3.74999795 -3.74992208 -3.75
 -3.75       -3.5       ]

#### `Simulation.dust.q.sweep`

This is the exponent of the particle size distribution in the drift limited case. By default, it is set to `-3`

In [34]:
sim.dust.q.sweep

-3.0

### `Simulation.dust.a`

Particle radii of shape `(Simulation.grid.Nr, 5)` calculated with

$[a_0, 0.4 * a_1, a_1, 0.4 * a_{max}, a_{max}]$ note that $a_{max}$ and $s_{max}$ are the same, so Simulation.dust.a[:,-1] == Simulation.dust.s.max

$a_0 = \left\{ \begin{aligned} \frac{s_{int}s_{min}}{s_{int}-s_{min}} \log\left(s_{int}/s_{min}\right) \textrm{ for }\xi = -5 \\ \frac{s_{int}-s_{min}}{\log\left(s_{int}/s_{min}\right)} \textrm{ for }\xi = -4 \\ \frac{\xi+4}{\xi+5} \frac{s_{int}^{\xi+5}-s_{min}^{\xi+5}}{s_{int}^{\xi+4}-s_{min}^{\xi+4}} \textrm{ for }\xi \neq -4, -5\end{aligned} \right.$

$a_1 = \left\{ \begin{aligned} \frac{s_{max}s_{int}}{s_{max}-s_{int}} \log\left(s_{max}/s_{int}\right) \textrm{ for }\xi = -5 \\ \frac{s_{max}-s_{int}}{\log\left(s_{max}/s_{int}\right)} \textrm{ for }\xi = -4 \\ \frac{\xi+4}{\xi+5} \frac{s_{max}^{\xi+5}-s_{int}^{\xi+5}}{s_{max}^{\xi+4}-s_{int}^{\xi+4}} \textrm{ for }\xi \neq -4, -5\end{aligned} \right.$


### `Simulation.dust.D`

Dust diffusivity for every particle species of shape `(Simulation.grid.Nr, 5)` calculated with

$D = \frac{\delta_\mathrm{rad}\,c_\mathrm{s}^2}{\Omega_\mathrm{K}\left( 1 + \mathrm{St}^2 \right)}$.

### `Simulation.dust.eps`

This is the vertically integrated dust-to-gas ratio of shape `(Simulation.grid.Nr)`. In the literature this is also often refered to as metallicity $z$. It is calculated via

$\epsilon = \frac{\sum\limits_i \Sigma_{\mathrm{d},\,i}}{\Sigma_\mathrm{g}}$

### `Simulation.dust.fill`

This describes the filling factor of the dust aggregates. By default this is 1.

<span style="color: red;"> NOTE: do not make this size dependent otherwies the TriPoD method breaks down</span>


this is because the method assumes the dust number size distribution and dust mass size distribution relate as $ \sigma(a) \propto n(a) a^3 \rightarrow \rho(a) = \text{constant}$ so both the filling factor and bulk density should be kept independent of size. 

In [35]:
sim.dust.fill

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]

### `Simulation.dust.H`

These are the dust scale heights of shape `(Simulation.grid.Nr, 5)` calculated with the prescription of [Dubrulle et al. (1995)](https://doi.org/10.1006/icar.1995.1058)

$H_i = H_\mathrm{P} \cdot \sqrt{\frac{\delta_\mathrm{vert}}{\delta_\mathrm{vert}+\mathrm{St}_i}}$

It is limited to a maximum of the pressure scale height $H_\mathrm{P}$. It uses `Simulation.dust.delta.vert` as vertical mixing parameter instead of `Simulation.gas.alpha`.

### `Simulation.dust.m`

Particle masses of shape `(Simulation.grid.Nr, 5)` calculated with

$m = \frac{4}{3} \pi \rho_s a^3 * fill$

### `Simulation.dust.rho`

Midplane mass densities of the dust.

$\rho = \frac{\Sigma}{\sqrt{2\pi}H}$

### `Simulation.dust.rhos`

Solid state density of the particle material. This is initially set to `Simulation.ini.dust.rhoMonomer`.

this will be modified if you add mutiple active dust species -> see next notebook

In [36]:
sim.dust.rhos

[[1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]]

To calculate the mass density of a dust aggregate do

In [37]:
sim.dust.fill * sim.dust.rhos

[[1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]
 [1.67 1.67 1.67 1.67 1.67]]

This is the density that is used to calculate the particle size.

### `Simulation.dust.Sigma`

This is the dust surface density of the small and large particle species of shape `(Simulation.grid.Nr, 2)`. It is the quantity that is integrated to solve for dust evolution.

The surface densities are integrated over the mass bin, so a numerical summation over the mass dimension returns the total dust surface density.

$\Sigma_\mathrm{d} = \sum\limits_i \Sigma_{\mathrm{d},\,i}$

In [38]:
sim.dust.Sigma.sum(-1)

[9.55090227e+00 6.72228077e+00 4.70358204e+00 3.27523028e+00
 2.26511417e+00 1.55149499e+00 1.04832269e+00 6.94837882e-01
 4.48196716e-01 2.78215436e-01 1.63584842e-01 8.90919832e-02
 4.35456060e-02 1.82673318e-02 6.17516844e-03 2.00000000e-11
 2.00000000e-11 2.00000000e-11 2.00000000e-11 2.00000000e-11]

If you want to plot the dust density distribution, you might want to convert the field into a quantity that does not depend on the mass bin width. The plotting script integrated in `TwoPopPy` is plotting a grid independent density distribution. See the first chapter for details on this.

### `Simulation.dust.SigmaFloor`

This is the floor value for the dust surface densities. Mass bins that are below their respective floor value will not contribute to coagulation. By default, the floor value is the density that corresponds to one physical particle of that mass distributed over an annulus at that radial grid location.

$\Sigma_{\mathrm{d},\,\mathrm{floor}} = \frac{m}{A_\mathrm{annulus}}$

Densities below the floor value therefore correspond to fewer that one physical particle in that radial grid cell.

### `Simulation.dust.St`

Stokes numbers of the particles. By default the Epstein and the Stokes I drag regime are considered. Has shape `(Simulation.grid.Nr,5)` i.e. it is defined for each size in the `a` array.

$\mathrm{St} =
\begin{cases}
\frac{\pi}{2} \frac{a\rho}{\Sigma_\mathrm{g}}, & \text{if } a < \frac{9}{4} \lambda_\mathrm{mfp}\\
\frac{2\pi}{9} \frac{a^2 \rho}{\lambda_\mathrm{mfp} \Sigma_\mathrm{g}}, & \text{else}
\end{cases}$

In [39]:
sim.dust.St.shape

(20, 5)

### Update order

The update order of the dust quantities in the standard model is set to

In [40]:
sim.dust.updateorder

['delta',
 'rhos',
 'fill',
 'backreaction',
 'f',
 'qrec',
 'a',
 'm',
 'St',
 'H',
 'rho',
 'D',
 'eps',
 'v',
 'p',
 'q',
 'SigmaFloor',
 'S']

In [41]:
sim.dust.backreaction.updateorder

['A', 'B']

In [42]:
sim.dust.delta.updateorder

['rad', 'turb', 'vert']

In [43]:
sim.dust.Fi.updateorder

['adv', 'diff', 'tot']

In [44]:
sim.dust.p.updateorder

['frag', 'stick', 'fragtrans', 'driftfrag']

In [45]:
sim.dust.S.updateorder

['compo', 'ext', 'tot', 'smax_hyd']

In [46]:
sim.dust.s.updateorder

In [47]:
sim.dust.v.updateorder

['frag', 'driftmax', 'rel']

In [48]:
sim.dust.v.rel.updateorder

['azi', 'brown', 'rad', 'turb', 'vert', 'tot']

In [49]:
sim.dust.q.updateorder

['frag', 'eff']

**Note**: The quantities that are excluded are calculated in the finalization step of the implicit integrator from the new values of the dust surface densities.

## Gas

In [50]:
sim.gas

Group (Gas quantities)
----------------------
    boundary     : Group (Boundary conditions)
    S            : Group (Source terms)
    torque       : Group (Torque parameters)
    v            : Group (Velocities)
  -----
    alpha        : Field (Turbulent alpha parameter)
    cs           : Field (Isothermal sound speed [cm/s])
    eta          : Field (Pressure gradient parameter)
    Fi           : Field (Gas flux interfaces [g/cm/s])
    Hp           : Field (Pressure scale height [cm])
    mfp          : Field (Midplane mean free path [cm])
    mu           : Field (Mean molecular weight [g])
    n            : Field (Miplane number density [1/cm³])
    nu           : Field (Kinematic viscosity [cm²/s])
    P            : Field (Midplane pressure [g/cm/s²])
    rho          : Field (Miplane mass density [g/cm³])
    Sigma        : Field (Surface density [g/cm²])
    SigmaFloor   : Field (Floor value of surface density [g/cm²])
    T            : Field (Temperature [K])
  -----

### `Simulation.gas.boundary`

In [51]:
sim.gas.boundary

Group (Boundary conditions)
---------------------------
    inner        : Boundary
    outer        : Boundary
  -----

These are the boundary conditions of the gas. By default the inner boundary is set to constant gradient and the outer boundary to floor value.

In [52]:
sim.gas.boundary.inner

Constant gradient

In [53]:
sim.gas.boundary.outer

Value

The boundary conditions can be modified with `setcondition`.

In [54]:
help(sim.gas.boundary.inner.setcondition)

Help on method setcondition in module dustpy.utils.boundary:

setcondition(condition, value=None) method of dustpy.utils.boundary.Boundary instance
    Function to set boundary condition.

    Parameters
    ----------
    condition : string
        Type of boundary conditon:
            - "const_grad" : constant gradient
            - "const_pow" : constant power law
            - "const_val" : constant value
            - "val" : custom value
            - "grad" : custom gradient
            - "pow" : custom power law with set exponent
            - None : Don't impose boundary condition (default)
    value : float or array, optional, default : None
        Value if needed for boundary condition



If the gas surface density follows a power law $\propto R^{-1}$ the constant gradient boundary condition should work fine. Other values can lead to deviations at the inner boundary. See the chapter about gas evolution tests for details.

### `Simulation.gas.S`

In [55]:
sim.gas.S

Group (Source terms)
--------------------
    ext          : Field (External sources [g/cm²/s])
    hyd          : Field (Hydrodynamic sources [g/cm²/s])
    tot          : Field (Total sources [g/cm²/s])
  -----

These are the source terms of the gas.

#### `Simulation.gas.S.ext`

These are the external source terms for gas evolution, e.g. infall. By default these are set to zero.

In [56]:
sim.gas.S.ext

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

#### `Simulation.gas.S.hyd`

These are the hydrodynamic source terms of the gas evolution.  
**Attention:** Since the gas evolution is calculated implicitly, the hydrodynamic source terms are calculated in retrospect after the new gas surface density was found. Changing `Simulation.gas.S.hyd` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.S.tot`

These are the total source terms of gas evolution.

$S_\mathrm{tot} = S_\mathrm{ext} + S_\mathrm{hyd}$

**Attention:** Since the gas evolution is calculated implicitly, the total source terms are calculated in retrospect after the new gas surface density was found. Changing `Simulation.gas.S.tot` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.v`

In [57]:
sim.gas.v

Group (Velocities)
------------------
    rad          : Field (Radial velocity [cm/s])
    visc         : Field (Viscous accretion velocity [cm/s])
  -----

These are velocities that are relevant for the gas evolution.  
**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.v.rad`

This is the radial gas velocity. It is given by

$v_\mathrm{g} = Av_\mathrm{visc} + 2B\eta v_\mathrm{K}$.

See `Simulation.dust.backreaction` for details. If backreaction is turned off, i.e., $A=1$ and $B=0$, the radial velocity is identical to the viscous velocity.  
**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v.rad` does not influence the simulation. It is only given for data analysis.

#### `Simulation.gas.v.visc`

This is the radial viscous gas velocity

$v_\mathrm{visc} = -\frac{3}{\Sigma_\mathrm{g}\sqrt{R}} \frac{\partial}{\partial R} \left( \Sigma_\mathrm{g} \nu \sqrt{R} \right)$

**Attention:** Since the gas evolution is calculated implicitly, the velocities are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.v.visc` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.alpha`

This is the turbulent viscosity parameter according [Shakura & Sunyaev (1973)](https://ui.adsabs.harvard.edu/abs/1973A%26A....24..337S/abstract). It is initially set to the value in `Simulation.ini.gas.alpha`.

In [58]:
sim.gas.alpha

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]

### `Simulation.gas.cs`

This is the adiabatic sound speed in the midplane of the disk

$c_\mathrm{s} = \sqrt{\frac{k_\mathrm{B} T}{\mu}}$.


### `Simulation.gas.eta`

This is the midplane pressure gradient parameter $\eta$ given by

$\eta = -\frac{1}{2} \left( \frac{H_\mathrm{P}}{r} \right)^2 \frac{\partial \log P}{\partial \log r}$

It describes the degree of "sub-Keplerity" of the disk

$v_\phi^2 = \left( 1-2\eta \right) v_\mathrm{K}^2$.

### `Simulation.gas.Fi`

These are the mass fluxes of gas through the grid cell interfaces.  
**Attention:** Since the gas evolution is calculated implicitly, the fluxes are calculated in retrospect after the new gas surface density was found. Changing anything in `Simulation.gas.Fi` does not influence the simulation. It is only given for data analysis.

### `Simulation.gas.Hp`

Pressure scale height of the gas given by the ratio of the isothermal sound speed to the Keplerian frequency.

$H_\mathrm{P} = \frac{c_{\mathrm{s},\,\mathrm{iso}}}{\Omega_\mathrm{K}}$.

### `Simulation.gas.mfp`

Mean free path of the gas in the midplane of the disk

$\lambda_\mathrm{mfp} = \frac{1}{\sqrt{2}\,n\,\sigma_\mathrm{H_2}}$

### `Simulation.gas.mu`

Mean molecular weight of the gas. This is initially set by the value given in `Simulation.ini.gas.mu` and is equal to $2.3\,m_\mathrm{P}$.

In [59]:
sim.gas.mu

[3.84703042e-24 3.84703042e-24 3.84703042e-24 3.84703042e-24
 3.84703042e-24 3.84703042e-24 3.84703042e-24 3.84703042e-24
 3.84703042e-24 3.84703042e-24 3.84703042e-24 3.84703042e-24
 3.84703042e-24 3.84703042e-24 3.84703042e-24 3.84703042e-24
 3.84703042e-24 3.84703042e-24 3.84703042e-24 3.84703042e-24]

### `Simulation.gas.n`

Midplane number density of the gas given by

$n = \frac{\rho}{\mu}$

### `Simulation.gas.nu`

Kinematic viscosity of the gas given by

$\nu = \alpha c_\mathrm{s} H_\mathrm{P}$.

### `Simulation.gas.P`

Midplane gas pressure given by

$P = \frac{\rho\,c_\mathrm{s}^2}{\gamma}$.

### `Simulation.gas.rho`

Midplane gas mass density given by

$\rho = \frac{\Sigma_\mathrm{g}}{\sqrt{2\pi}H_\mathrm{P}}$.

### `Simulation.gas.Sigma`

Gas surface density. This is the quantity that is integrated with an implicit Euler first-order scheme.

### `Simulation.gas.SigmaFloor`

This is the floor value of the gas surface density. By default it is $10^{-100}$.

In [60]:
sim.gas.SigmaFloor

[1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100 1.e-100
 1.e-100 1.e-100]

If the gas surface density is at any point below it's floor value it will be automatically set to the floor value at the end of a time step.

### `Simulation.gas.T`

This is the midplane gas temperature. It is calculated by assuming a passively irradiated disk with a constant irradiation angle of $0.05$.

$T\left( r \right) = \sqrt[4]{\frac{0.05\,L_*}{4\,\pi\,r^2\,\sigma_\mathrm{SB}}}$

### Update order

The update order of the gas quantities in the standard model is set to

In [61]:
sim.gas.updateorder

['Sigma',
 'mu',
 'T',
 'alpha',
 'cs',
 'Hp',
 'nu',
 'rho',
 'n',
 'mfp',
 'P',
 'eta',
 'torque',
 'S']

In [62]:
sim.gas.S.updateorder

['ext', 'tot']

In [63]:
sim.gas.v.updateorder

['visc', 'rad']

**Note**: The quantities that are excluded are calculated in the finalization step of the implicit integrator from the new values of the surface densities.

## Components

since we allow for multiple components to make up the gas disk i.e. different chemcial elements, we have a components group that gets initalized with one default component that makes up the entire gas in the disk

In [64]:
sim.components

Group (components)
------------------
    Default      : Group (Default gas componentDefault (dust_tracer=False, gas_active=True, gas_tracer=False))
  -----

This default component alsways has a gas and dust group(which is empty for this default component)

In [65]:
sim.components.Default

Group (Default gas componentDefault (dust_tracer=False, gas_active=True, gas_tracer=False))
-------------------------------------------------------------------------------------------
    boundary     : Group (Boundary conditions for component)
    dust         : Group (Dust related fields)
    gas          : Group (Gas related fields)
  -----

each component has its gas and dust contents. For the Default only gas is active (further functionality of components will be explaine on the next page)

In [66]:
sim.components.Default.gas

Group (Gas related fields)
--------------------------
    boundary     : Group
    pars         : Group (Gas parameters)
    S            : Group (Gas source terms)
  -----
    Fi           : Field (Gas flux [g/cm²/s])
    Sigma        : Field (Gas surface density [g/cm²])
    Sigma_dot    : Field (Gas surface density source term [g/cm²/s])
    value        : Field (Gas parameter [???])
    value_dot    : Field (Gas parameter source term [???/s])
  -----

### `Simulation.components.Default.gas.Sigma`

The surface desity of this specific gas species that contributes to the total Simulation.gas.Sigma

In [67]:
all(sim.components.Default.gas.Sigma == sim.gas.Sigma)

True

### `Simulation.components.Default.gas.Sigma_dot`

The extenal source term of Simulation.components.Default.gas.Sigma

In [68]:
sim.components.Default.gas.Sigma_dot

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

### `Simulation.components.Default.gas.Fi`

the radial flux of this component at the cell interfaces

In [69]:
sim.components.Default.gas.Fi

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

### `Simulation.components.Default.gas.value`

if the component is set as a tracer tracking a property (e.g. gas that originated from a specific location), the value tracks the distribution of the tracer (e.g. the local density of gas in the disc from one original location after time) and value_dot is its external source term.

In [70]:
sim.components.Default.gas.value

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

### `Simulation.components.Default.S`

Group containing the source terms of this components that are used for the time step calculation. The fields of the source terms for each components have the same structure and function as the source terms covered in `Simulation.gas.S` for the total gas.



In [71]:
sim.components.Default.gas.S

Group (Gas source terms)
------------------------
    ext          : Field (External source term [g/cm²/s])
    hyd          : Field (Hydrodynamical source term [g/cm²/s])
    tot          : Field (Total source term [g/cm²/s])
  -----

### `Simulation.components.Default.pars.mu`

the mean molecular weight of this gas component 

In [72]:
sim.components.Default.gas.pars.mu

3.847030424486999e-24

## Grid

In [73]:
sim.grid

Group (Grid quantities)
-----------------------
    A            : Field (Radial grid annulus area [cm²]), constant
    Nr           : Field (# of radial grid cells), constant
    OmegaK       : Field (Keplerian frequency [1/s])
    r            : Field (Radial grid cell centers [cm]), constant
    ri           : Field (Radial grid cell interfaces [cm]), constant
  -----

These are all quantities that define the radial and the mass grid. Once they are defined they are constant and should not be changed. Additionally, the Keplerian frequency is located here.

### `Simulation.grid.A`

Surface area of the annulus a grid cell spans.

$A_i = \pi \left( r_{i+\frac{1}{2}}^2 - r_{i-\frac{1}{2}}^2 \right)$

### `Simulation.grid.Nr`

Number of radial grid cells.

In [74]:
sim.grid.Nr

20

### `Simulation.grid.OmegaK`

Keplerian frequency given by

$\Omega_\mathrm{K} = \sqrt{\frac{G\,M_*}{r^3}}$.

### `Simulation.grid.r`

Radial grid cell centers. The radial grid cell centers are exactly in the middle between the grid cell interfaces.

In [75]:
sim.grid.r == 0.5 * (sim.grid.ri[:-1] + sim.grid.ri[1:])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]

### `Simulation.grid.ri`

Locations of the grid cell interfaces. `Simulation.grid.ri[0]` and `Simulation.grid.ri[-1]` are the inner and outer grid boundaries.

### Update order

The update order of the grid quantities in the standard model is set to

In [76]:
sim.grid.updateorder

['OmegaK']

## Star

In [77]:
sim.star

Group (Stellar quantities)
--------------------------
    L            : Field (Luminosity [erg/s])
    M            : Field (Mass [g])
    R            : Field (Radius [cm])
    T            : Field (Effective temperature [K])
  -----

### `Simulation.star.L`

Stellar luminosity given by

$L = 4\pi\,R_*^2\,\sigma_\mathrm{SB}\,T_*^4$

### `Simulation.star.M`

Stellar mass. Initially set to the value given in `Simulation.ini.star.M`, which corresponds to one Solar mass.

In [78]:
sim.star.M

1.988409870698051e+33

### `Simulation.star.R`

Stellar radius. Initially set to the value given in `Simulation.ini.star.R`, which corresponds to 2 Solar radii.

In [79]:
sim.star.R

139140000000.0

### `Simulation.star.T`

Stellar effective surface temperature. Initially set to the value given in `Simulation.ini.star.T`.

In [80]:
sim.star.T

5772.0

### Update order

The update order of the stellar quantities in the standard model is set to

In [81]:
sim.star.updateorder

['M', 'R', 'T', 'L']

## Time

`Simulation.t` is the current time and the integration variable. It starts at zero initially.

In [82]:
sim.t

0.0

Snapshots are written between $10^3$ years and $10^5$ years with 10 snapshots per time decade.

In [83]:
sim.t.snapshots

array([3.15576000e+10, 3.97286646e+10, 5.00154254e+10, 6.29656900e+10,
       7.92691073e+10, 9.97938935e+10, 1.25633068e+11, 1.58162662e+11,
       1.99114995e+11, 2.50670927e+11, 3.15576000e+11, 3.97286646e+11,
       5.00154254e+11, 6.29656900e+11, 7.92691073e+11, 9.97938935e+11,
       1.25633068e+12, 1.58162662e+12, 1.99114995e+12, 2.50670927e+12,
       3.15576000e+12])

The timestep is calculated from the dust and the gas source terms and the current values of the surface densities of dust and gas, while using a safety factor of `0.1`, which can be accessed as an attribute of the integration variable.

In [84]:
sim.t.cfl

0.1

## Integrator

The integrator that is used by default as two integration instructions. One for the gas and one for the dust.

In [85]:
sim.integrator

Integrator (Default integrator)

In [86]:
sim.integrator.instructions

[Instruction (Dust (state vector): implicit 1st-order direct solver),
 Instruction (Update all int variable after dust step),
 Instruction (Default: implicit 1st-order direct solver for components),
 Instruction (Update Default)]

The gas is integrated with an implicit first-order Euler integration scheme. The Jacobian is calculated from the parameters given in `Simulation.gas`. Parameters like gas velocities, fluxes and source terms are calculated once the new values of the gas surface density have been found.

Except for the values at the boundaries, the advective source terms are given by

In [87]:
sim.gas.Sigma.jacobian() @ sim.gas.Sigma

array([ 0.00000000e+00, -1.29560708e-11, -6.19053581e-12, -4.25769931e-12,
       -2.89300213e-12, -1.93185055e-12, -1.25811498e-12, -7.90005205e-13,
       -4.69986127e-13, -2.57475635e-13, -1.23421390e-13, -4.61704653e-14,
       -8.40884154e-15,  4.64280071e-15,  5.40623139e-15,  2.72032079e-15,
        7.86313351e-16,  1.24366967e-16,  9.33737275e-18,  0.00000000e+00])

The total source terms, i.e., including external sources and excluding the boundaries are given by

In [88]:
sim.gas.Sigma.jacobian() @ sim.gas.Sigma + sim.gas.S.ext

[ 0.00000000e+00 -1.29560708e-11 -6.19053581e-12 -4.25769931e-12
 -2.89300213e-12 -1.93185055e-12 -1.25811498e-12 -7.90005205e-13
 -4.69986127e-13 -2.57475635e-13 -1.23421390e-13 -4.61704653e-14
 -8.40884154e-15  4.64280071e-15  5.40623139e-15  2.72032079e-15
  7.86313351e-16  1.24366967e-16  9.33737275e-18  0.00000000e+00]

The dust is integrated with an implicit first-order Euler integration scheme. The Jacobian is calculated from the parameters given in `Simulation.dust`. Parameters like gas velocities, fluxes and source terms are calculated once the new values of the gas surface density have been found.

Except for the values at the boundaries, the hydrodynamic and coagulation source terms are given by

In [89]:
(sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel()).reshape(sim.dust.Sigma.shape)

array([[ 1.38279410e-10,  1.62614003e-10],
       [-1.89561639e-09,  1.89561639e-09],
       [-5.57161851e-10,  5.57161851e-10],
       [-1.63108791e-10,  1.63108791e-10],
       [-4.75667730e-11,  4.75667730e-11],
       [-1.38371790e-11,  1.38371790e-11],
       [-4.02822610e-12,  4.02822610e-12],
       [-1.17937750e-12,  1.17937750e-12],
       [-3.48579297e-13,  3.48579297e-13],
       [-1.03584690e-13,  1.03584690e-13],
       [-3.03765737e-14,  3.03765737e-14],
       [-8.49589508e-15,  8.49589508e-15],
       [-2.16493449e-15,  2.16493449e-15],
       [-4.75196365e-16,  4.75196365e-16],
       [-1.03672523e-16,  5.96629378e-17],
       [ 1.01365967e-17,  1.19204462e-17],
       [ 1.99668178e-25,  1.99422280e-25],
       [ 1.08921153e-25,  1.08633110e-25],
       [-1.35317839e-25, -1.35097661e-25],
       [ 0.00000000e+00,  0.00000000e+00]])

Note the `ravel()` and `reshape()` operations, since the Jacobian works on a one-dimensional state vector.

The total source terms except at the boundaries are given by

In [90]:
(sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel()).reshape(sim.dust.Sigma.shape) + sim.dust.S.ext

[[ 1.38279410e-10  1.62614003e-10]
 [-1.89561639e-09  1.89561639e-09]
 [-5.57161851e-10  5.57161851e-10]
 [-1.63108791e-10  1.63108791e-10]
 [-4.75667730e-11  4.75667730e-11]
 [-1.38371790e-11  1.38371790e-11]
 [-4.02822610e-12  4.02822610e-12]
 [-1.17937750e-12  1.17937750e-12]
 [-3.48579297e-13  3.48579297e-13]
 [-1.03584690e-13  1.03584690e-13]
 [-3.03765737e-14  3.03765737e-14]
 [-8.49589508e-15  8.49589508e-15]
 [-2.16493449e-15  2.16493449e-15]
 [-4.75196365e-16  4.75196365e-16]
 [-1.03672523e-16  5.96629378e-17]
 [ 1.01365967e-17  1.19204462e-17]
 [ 1.99668178e-25  1.99422280e-25]
 [ 1.08921153e-25  1.08633110e-25]
 [-1.35317839e-25 -1.35097661e-25]
 [ 0.00000000e+00  0.00000000e+00]]

At the end of a successful integration step, the floor values and boundaries are enforced.

the dutst integrator works much in the same way using a first oreder implicit scheme to update $\Sigma_0$, $\Sigma_1$ and $s_{max}$ at the same time:

In [91]:
sim.dust.Sigma.jacobian() @ sim.dust.Sigma.ravel() + sim.dust.S.ext.ravel()

[ 1.38279410e-10  1.62614003e-10 -1.89561639e-09  1.89561639e-09
 -5.57161851e-10  5.57161851e-10 -1.63108791e-10  1.63108791e-10
 -4.75667730e-11  4.75667730e-11 -1.38371790e-11  1.38371790e-11
 -4.02822610e-12  4.02822610e-12 -1.17937750e-12  1.17937750e-12
 -3.48579297e-13  3.48579297e-13 -1.03584690e-13  1.03584690e-13
 -3.03765737e-14  3.03765737e-14 -8.49589508e-15  8.49589508e-15
 -2.16493449e-15  2.16493449e-15 -4.75196365e-16  4.75196365e-16
 -1.03672523e-16  5.96629378e-17  1.01365967e-17  1.19204462e-17
  1.99668178e-25  1.99422280e-25  1.08921153e-25  1.08633110e-25
 -1.35317839e-25 -1.35097661e-25  0.00000000e+00  0.00000000e+00]

Note that growth term of $s_{max}$ has to be treated explicitly since it is highly non linear in the integation variables

## Writer

`TriPoDPy` uses by default the `hdf5writer` of `simframe`.

In [92]:
sim.writer

Writer (HDF5 file format using h5py)
------------------------------------
    Data directory : data
    File names     : data/data0000.hdf5
    Overwrite      : False
    Dumping        : True
    Options        : {'com': 'lzf', 'comopts': None}
    Verbosity      : 1